In [1]:
import dspy
from dspy.predict import Retry
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
import importlib
import json

importlib.reload(dspy)

# Configure the AzureOpenAI language model
azure_turbo = dspy.AzureOpenAI(api_base='https://bionlp-gpt4-wang.openai.azure.com/', api_key="a494edc84d714b6c8a12e7212974b793", api_version='2024-03-01-preview', model='gpt-35-turbo')
dspy.settings.configure(lm=azure_turbo)

/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LINE 127
System Prompt: None


In [2]:
class PubMedQA(dspy.Signature):
    question = dspy.InputField(desc="Question")
    context = dspy.InputField(desc="A list of facts which provide the necessary context to answer the question. You should directly mention the portions of the facts that you used to help answer the question.")
    options = dspy.InputField(desc="Options, you should select one of them based on whichever is correct.")
    answer = dspy.OutputField(desc="Your final answer should contain only one of YES, MAYBE, or NO (all in upper case) based on whichever option is correct")


class PubMedQA_Response(dspy.Module):
    """Use the provided context to answer a question by providing a step-by-step explanation and also respond with one of YES, MAYBE, or NO (all in upper case) based on whichever is correct"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(PubMedQA)

    def forward(self, question, context, options):
        response = self.generate_answer(context=context, question=question, options=options)

        '''
        while len(response.completions.answer) < 5:

            curr_len = len(response.completions.answer)

            for i in range(curr_len):
                response.completions.answer.append(response.completions.answer[i])
                response.completions.rationale.append(response.completions.rationale[i])

                if len(response.completions.answer) == 5:
                    break
        '''
        #best_response = self.compare_answers(context=context, question=question, options=options, completions=response.completions)
        
        #valid_response = "YES" in response.answer or "MAYBE" in response.answer or "NO" in response.answer
        #dspy.Suggest(valid_response, "Your final answer must be one of YES, MAYBE, or NO (all upper case) as part of your final answer")
        
        return dspy.Prediction(answer=response.answer, rationale=response.rationale)

In [3]:
import re

def eval_metric_pubmedqa(true, prediction, trace=None):
    try:
        pred = prediction.answer
        matches = re.findall(r"\b(YES|NO|MAYBE)\b", pred)
        parsed_answer = matches[-1] if matches else ""
        return parsed_answer == true.answer
    except:
        return False

In [4]:
import json 


with open("/Users/khandekarns/Documents/pubmedqa/data/test_set.json") as file:
    test_set = json.load(file)


with open("/Users/khandekarns/Documents/pubmedqa/data/test_ground_truth.json") as file:
    gt = json.load(file)


dspy_test_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in test_set.items():

    combined_context = ""

    for i, context in enumerate(test_set[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": test_set[key]['QUESTION'], "options": options, "context": combined_context, "answer": gt[key].upper()}).with_inputs("question", "context", "options") 
    dspy_test_set_pubmedqa.append(example)

In [5]:
import random

random.seed(42)

count = 0

with open("/Users/khandekarns/Documents/pubmedqa/data/ori_pqal.json") as file:
    val_data = json.load(file)

print(len(val_data))


dspy_val_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in val_data.items():

    if key in test_set:
        count += 1
        continue

    combined_context = ""

    for i, context in enumerate(val_data[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": val_data[key]['QUESTION'], "options": options, "context": combined_context, "answer": val_data[key]['final_decision'].upper()}).with_inputs("question", "context", "options") 

    dspy_val_set_pubmedqa.append(example)


# Shuffle the examples
random.shuffle(dspy_val_set_pubmedqa)

# Split into training and validation sets
dspy_train_set_pubmedqa = dspy_val_set_pubmedqa[:450]
dspy_val_set_pubmedqa = dspy_val_set_pubmedqa[450:500]

1000


In [6]:
from dspy.evaluate import Evaluate

evaluate_test = Evaluate(devset=dspy_test_set_pubmedqa, metric=eval_metric_pubmedqa, num_threads=3, display_progress=True, display_table=True)

In [9]:
#answerqa_with_assertions = assert_transform_module(PubMedQA_Response().map_named_predictors(Retry), backtrack_handler) 
evaluate_test(PubMedQA_Response())

Average Metric: 3 / 3  (100.0):   1%|          | 3/500 [00:05<11:21,  1.37s/it]/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/joblib/_store_backends.py:215: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: [Errno 2] No such file or directory: '/Users/khandekarns/cachedir_joblib/joblib/dsp/modules/azure_openai/v1_chat_request/<locals>/v1_cached_gpt3_turbo_request_v2_wrapped/<locals>/v1_cached_gpt3_turbo_request_v2/b818c90bda6aeb6401a7a4289f9687f3/output.pkl.thread-5367005792-pid-65869' -> '/Users/khandekarns/cachedir_joblib/joblib/dsp/modules/azure_openai/v1_chat_request/<locals>/v1_cached_gpt3_turbo_request_v2_wrapped/<locals>/v1_cached_gpt3_turbo_request_v2/b818c90bda6aeb6401a7a4289f9687f3/output.pkl'.
  warnings.warn(
Average Metric: 111 / 125  (88.8):  25%|██▌       | 125/500 [01:24<05:08,  1.22it/s]/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/joblib/_store_backends.py:215: CacheWarning: Unable to

Backing off 0.6 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 358 / 500  (71.6): 100%|██████████| 500/500 [05:18<00:00,  1.57it/s]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Is anorectal endosonography valuable in dyschesia?,1. YES 2. MAYBE 3. NO,1. Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the...,YES,YES,produce the answer. 1. The study aimed to demonstrate dysfunction of the anal sphincter and/or the musculus (m.) puborectalis in patients with dyschesia using anorectal...,✔️ [True]
1,Is there a connection between sublingual varices and hypertension?,1. YES 2. MAYBE 3. NO,"1. Sublingual varices have earlier been related to ageing, smoking and cardiovascular disease. The aim of this study was to investigate whether sublingual varices are...",YES,YES,"produce the answer. 1. The context mentions that sublingual varices have been related to cardiovascular disease, which includes hypertension. This suggests that there might be...",✔️ [True]
2,Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?,1. YES 2. MAYBE 3. NO,"1. Tacrolimus is a potent immunosuppressive drug used in organ transplantation. Because of its substantial toxic effects, narrow therapeutic index, and interindividual pharmacokinetic variability, therapeutic...",YES,YES,produce the answer. 1. The question asks if the affinity column-mediated immunoassay method is suitable as an alternative to the microparticle enzyme immunoassay method for...,✔️ [True]
3,Does a physician's specialty influence the recording of medication history in patients' case notes?,1. YES 2. MAYBE 3. NO,1. To determine the impact of a physician's specialty on the frequency and depth of medication history documented in patient medical records. 2. A cross-sectional...,YES,YES,produce the answer. 1. We are given a study that assessed the frequency and depth of medication history documentation by physicians in different specialties. 2....,✔️ [True]
4,Locoregional opening of the rodent blood-brain barrier for paclitaxel using Nd:YAG laser-induced thermo therapy: a new concept of adjuvant glioma therapy?,1. YES 2. MAYBE 3. NO,1. Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes. We address the question of whether LITT-induced locoregional...,YES,YES,produce the answer. 1. The study conducted Nd:YAG laser-induced thermo therapy (LITT) on rat brains. 2. The researchers analyzed the disruption of the blood-brain barrier...,✔️ [True]
5,Spinal subdural hematoma: a sequela of a ruptured intracranial aneurysm?,1. YES 2. MAYBE 3. NO,1. A case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm is described. Such a case has...,YES,YES,produce the answer. 1. The context mentions a case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm....,✔️ [True]
6,Is there a correlation between androgens and sexual desire in women?,1. YES 2. MAYBE 3. NO,"1. For women, the correlation between circulating androgens and sexual desire is inconclusive. Substitution with androgens at physiological levels improves sexual function in women who...",YES,YES,"produce the answer. 1. We are given that the correlation between circulating androgens and sexual desire in women is inconclusive. 2. However, substitution with androgens...",✔️ [True]
7,Is the zeolite hemostatic agent beneficial in reducing blood loss during arterial injury?,1. YES 2. MAYBE 3. NO,1. Uncontrolled hemorrhage is the leading cause of fatality. The aim of this study was to evaluate the effect of zeolite mineral (QuikClot - Advanced...,YES,YES,produce the answer. We have a study that evaluated the effect of zeolite mineral (QuikClot - Advanced Clotting Sponge [QC-ACS]) on blood loss during arterial...,✔️ [True]
8,Are endothelial 

71.6

In [10]:
from tqdm import tqdm
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

tqdm._instances.clear()

config = dict(max_bootstrapped_demos=5, max_labeled_demos=0, num_candidate_programs=20, num_threads=6)
teleprompter = BootstrapFewShotWithRandomSearch(metric = eval_metric_pubmedqa, **config)
answer_question_teacher = teleprompter.compile(PubMedQA_Response(), trainset=dspy_train_set_pubmedqa, valset=dspy_val_set_pubmedqa)

Going to sample between 1 and 5 traces per predictor.
Will attempt to bootstrap 20 candidate sets.


Average Metric: 3 / 6  (50.0):  12%|█▏        | 6/50 [00:04<00:33,  1.30it/s] /opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/joblib/_store_backends.py:215: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: [Errno 22] Invalid argument: '/Users/khandekarns/cachedir_joblib/joblib/dsp/modules/azure_openai/v1_chat_request/<locals>/v1_cached_gpt3_turbo_request_v2_wrapped/<locals>/v1_cached_gpt3_turbo_request_v2/e7b6101a988c1cf4254fcae2db8d997b'.
  warnings.warn(
Average Metric: 5 / 8  (62.5):  16%|█▌        | 8/50 [00:04<00:19,  2.15it/s]INFO:backoff:Backing off request(...) for 0.8s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would lik

Backing off 0.8 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'max_tokens': 75, 'n': 1, 'temperature': 0.0}


Average Metric: 9 / 13  (69.2):  26%|██▌       | 13/50 [00:07<00:16,  2.31it/s]INFO:backoff:Backing off request(...) for 0.4s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.4 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'max_tokens': 75, 'n': 1, 'temperature': 0.0}


...

Backing off 0.5 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 37 / 45  (82.2):  90%|█████████ | 45/50 [00:16<00:01,  3.87it/s]INFO:backoff:Backing off request(...) for 0.4s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.4 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 40 / 50  (80.0): 100%|██████████| 50/50 [00:19<00:00,  2.51it/s]

Score: 80.0 for set: [1]
Scores so far: [74.0, 74.0, 62.0, 72.0, 70.0, 74.0, 72.0, 68.0, 68.0, 80.0, 68.0, 74.0, 80.0, 72.0, 76.0, 68.0, 76.0, 82.0, 70.0, 72.0, 68.0, 58.0, 80.0]
Best score: 82.0
Average of max per entry across top 1 scores: 0.82
Average of max per entry across top 2 scores: 0.88
Average of max per entry across top 3 scores: 0.9
Average of max per entry across top 5 scores: 0.92
Average of max per entry across top 8 scores: 0.96
Average of max per entry across top 9999 scores: 0.98
23 candidate programs found.


In [12]:
evaluate_test(answer_question_teacher)

Average Metric: 76 / 90  (84.4):  18%|█▊        | 90/500 [00:55<05:14,  1.30it/s]INFO:backoff:Backing off request(...) for 0.8s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.8 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 149 / 179  (83.2):  36%|███▌      | 179/500 [01:52<04:02,  1.32it/s]INFO:backoff:Backing off request(...) for 0.2s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.2 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 236 / 283  (83.4):  57%|█████▋    | 283/500 [02:58<02:45,  1.31it/s]

...

Average Metric: 358 / 471  (76.0):  94%|█████████▍| 471/500 [05:03<00:40,  1.39s/it]INFO:backoff:Backing off request(...) for 2.3s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 2.3 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 361 / 500  (72.2): 100%|██████████| 500/500 [05:21<00:00,  1.55it/s]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Is anorectal endosonography valuable in dyschesia?,1. YES 2. MAYBE 3. NO,1. Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the...,YES,YES,produce the answer. 1. The question asks whether anorectal endosonography is valuable in dyschesia. 2. The context provides information about a prospective study that aimed...,✔️ [True]
1,Is there a connection between sublingual varices and hypertension?,1. YES 2. MAYBE 3. NO,"1. Sublingual varices have earlier been related to ageing, smoking and cardiovascular disease. The aim of this study was to investigate whether sublingual varices are...",YES,YES,produce the answer. 1. The question asks whether there is a connection between sublingual varices and hypertension. 2. The context describes a study that investigated...,✔️ [True]
2,Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?,1. YES 2. MAYBE 3. NO,"1. Tacrolimus is a potent immunosuppressive drug used in organ transplantation. Because of its substantial toxic effects, narrow therapeutic index, and interindividual pharmacokinetic variability, therapeutic...",YES,YES,produce the answer. 1. The question asks whether the affinity column-mediated immunoassay method is suitable as an alternative to the microparticle enzyme immunoassay method for...,✔️ [True]
3,Does a physician's specialty influence the recording of medication history in patients' case notes?,1. YES 2. MAYBE 3. NO,1. To determine the impact of a physician's specialty on the frequency and depth of medication history documented in patient medical records. 2. A cross-sectional...,YES,YES,produce the answer. 1. The question asks whether a physician's specialty influences the recording of medication history in patients' case notes. 2. The context describes...,✔️ [True]
4,Locoregional opening of the rodent blood-brain barrier for paclitaxel using Nd:YAG laser-induced thermo therapy: a new concept of adjuvant glioma therapy?,1. YES 2. MAYBE 3. NO,1. Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes. We address the question of whether LITT-induced locoregional...,YES,YES,produce the answer. 1. The question asks whether locoregional opening of the rodent blood-brain barrier (BBB) using Nd:YAG laser-induced thermo therapy (LITT) could be a...,✔️ [True]
5,Spinal subdural hematoma: a sequela of a ruptured intracranial aneurysm?,1. YES 2. MAYBE 3. NO,1. A case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm is described. Such a case has...,YES,NO,produce the answer. 1. The question asks whether spinal subdural hematoma (SSDH) is a sequela of a ruptured intracranial aneurysm. 2. The context describes a...,False
6,Is there a correlation between androgens and sexual desire in women?,1. YES 2. MAYBE 3. NO,"1. For women, the correlation between circulating androgens and sexual desire is inconclusive. Substitution with androgens at physiological levels improves sexual function in women who...",YES,NO,produce the answer. 1. The question asks whether there is a correlation between androgens and sexual desire in women. 2. The context provides information about...,False
7,Is the zeolite hemostatic agent beneficial in reducing blood loss during arterial injury?,1. YES 2. MAYBE 3. NO,1. Uncontrolled hemorrhage is the leading cause of fatality. The aim of this study was to evaluate the effect of zeolite mineral (QuikClot - Advanced...,YES,YES,produce the answer. 1. The question asks whether the zeolite hemostatic agent is beneficial in reducing blood loss during arterial injury. 2. The context provides...,✔️ [True]
8,Are endothelial cell patterns of astrocytomas

72.2

In [ ]:
azure_turbo.inspect_history()

In [17]:
from dspy.teleprompt import MIPROv2
import random


teleprompter = MIPROv2(prompt_model= azure_turbo, task_model=azure_turbo, metric=eval_metric_pubmedqa, num_candidates=5, metric_threshold=None)
kwargs = dict(num_threads=3, display_progress=True, display_table=True)
compiled_prompt_opt = teleprompter.compile(PubMedQA_Response(), trainset=dspy_train_set_pubmedqa, valset= dspy_val_set_pubmedqa, max_bootstrapped_demos=3, max_labeled_demos=0, eval_kwargs=kwargs, requires_permission_to_run=False)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 5 * 1 lm calls in program + (2) lm calls in program aware proposer = 17 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 450 examples in train set * 3 full evals = 2100 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may 

INFO:backoff:Backing off request(...) for 0.4s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4104 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 0.4 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'n': 1, 'temperature': 1.0}


INFO:backoff:Backing off request(...) for 0.3s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4104 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 0.3 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'n': 1, 'temperature': 1.0}


...

Backing off 3.5 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'n': 1, 'temperature': 1.0}


INFO:backoff:Backing off request(...) for 0.1s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4104 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
INFO:backoff:Backing off request(...) for 4.2s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4104 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 0.1 seconds after 4 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'n': 1, 'temperature': 1.0}
Backing off 4.2 seconds after 5 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {'n': 1, 'temperature': 1.0}


ERROR:backoff:Giving up request(...) after 6 tries (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4104 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4104 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
evaluate_test(compiled_prompt_opt)

Average Metric: 68 / 82  (82.9):  16%|█▋        | 82/500 [00:49<04:51,  1.43it/s]/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/joblib/_store_backends.py:215: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: [Errno 2] No such file or directory: '/Users/khandekarns/cachedir_joblib/joblib/dsp/modules/azure_openai/v1_chat_request/<locals>/v1_cached_gpt3_turbo_request_v2_wrapped/<locals>/v1_cached_gpt3_turbo_request_v2/199594f63e585e88715fbd5ac062997d/output.pkl.thread-5396434288-pid-65869' -> '/Users/khandekarns/cachedir_joblib/joblib/dsp/modules/azure_openai/v1_chat_request/<locals>/v1_cached_gpt3_turbo_request_v2_wrapped/<locals>/v1_cached_gpt3_turbo_request_v2/199594f63e585e88715fbd5ac062997d/output.pkl'.
  warnings.warn(
Average Metric: 192 / 231  (83.1):  46%|████▌     | 231/500 [02:12<03:35,  1.25it/s]INFO:backoff:Backing off request(...) for 0.7s (openai.RateLimitError: Error code: 429 - {'error': {'code': '42

Backing off 0.7 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1172d6ac0> with kwargs {}


Average Metric: 366 / 500  (73.2): 100%|██████████| 500/500 [04:52<00:00,  1.71it/s]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Is anorectal endosonography valuable in dyschesia?,1. YES 2. MAYBE 3. NO,1. Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the...,YES,YES,produce the answer. 1. The study aimed to demonstrate dysfunction of the anal sphincter and/or the musculus (m.) puborectalis in patients with dyschesia using anorectal...,✔️ [True]
1,Is there a connection between sublingual varices and hypertension?,1. YES 2. MAYBE 3. NO,"1. Sublingual varices have earlier been related to ageing, smoking and cardiovascular disease. The aim of this study was to investigate whether sublingual varices are...",YES,YES,produce the answer. 1. The study aimed to investigate whether sublingual varices are related to the presence of hypertension. 2. The study included 431 dental...,✔️ [True]
2,Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?,1. YES 2. MAYBE 3. NO,"1. Tacrolimus is a potent immunosuppressive drug used in organ transplantation. Because of its substantial toxic effects, narrow therapeutic index, and interindividual pharmacokinetic variability, therapeutic...",YES,YES,"produce the answer. 1. The study compared the results of two immunoassay systems, affinity column-mediated immunoassay (ACMIA) and microparticle enzyme immunoassay (MEIA), for measuring tacrolimus...",✔️ [True]
3,Does a physician's specialty influence the recording of medication history in patients' case notes?,1. YES 2. MAYBE 3. NO,1. To determine the impact of a physician's specialty on the frequency and depth of medication history documented in patient medical records. 2. A cross-sectional...,YES,YES,produce the answer. 1. The study aimed to determine the impact of a physician's specialty on the frequency and depth of medication history documented in...,✔️ [True]
4,Locoregional opening of the rodent blood-brain barrier for paclitaxel using Nd:YAG laser-induced thermo therapy: a new concept of adjuvant glioma therapy?,1. YES 2. MAYBE 3. NO,1. Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes. We address the question of whether LITT-induced locoregional...,YES,YES,produce the answer. 1. The study investigated whether Nd:YAG laser-induced thermo therapy (LITT) could disrupt the blood-brain barrier (BBB) and allow the passage of chemotherapeutic...,✔️ [True]
5,Spinal subdural hematoma: a sequela of a ruptured intracranial aneurysm?,1. YES 2. MAYBE 3. NO,1. A case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm is described. Such a case has...,YES,YES,produce the answer. 1. The case study describes a patient who developed a spinal subdural hematoma (SSDH) following a ruptured internal carotid aneurysm. 2. The...,✔️ [True]
6,Is there a correlation between androgens and sexual desire in women?,1. YES 2. MAYBE 3. NO,"1. For women, the correlation between circulating androgens and sexual desire is inconclusive. Substitution with androgens at physiological levels improves sexual function in women who...",YES,NO,produce the answer. 1. The study aimed to investigate the correlation between serum levels of androgens and sexual desire in women. 2. They measured various...,False
7,Is the zeolite hemostatic agent beneficial in reducing blood loss during arterial injury?,1. YES 2. MAYBE 3. NO,1. Uncontrolled hemorrhage is the leading cause of fatality. The aim of this study was to evaluate the effect of zeolite mineral (QuikClot - Advanced...,YES,YES,produce the answer. 1. The study aimed to evaluate the effect of zeolite mineral on blood loss and physiological variables in a swine extremity arterial...,✔️ [True]
8,Are endothelial cell patterns of astrocytomas

73.2